In [ ]:
#pip install lxml
#pip install openpyxl
#selenium
#pandas

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

# Danh sách ngân hàng và các tab dữ liệu
banks = ["VPB", "CTG", "VCB", "TCB", "MBB", "ACB", "STB", "HDB"]
tabs = [
    {"tab": "BCTT", "table_id": "tbl-data-BCTT-KQ"},
    {"tab": "CDKT", "table_id": "tbl-data-CDKT"},
    {"tab": "KQKD", "table_id": "tbl-data-KQKD"},
    {"tab": "LC",   "table_id": "tbl-data-LCTT-direct"},
    {"tab": "CSTC", "table_id": "tbl-data-CSTC"},
    {"tab": "TM",   "table_id": "tbl-data-TM"},
]

# Mở trình duyệt
driver = webdriver.Chrome()

# Đăng nhập
driver.get("https://finance.vietstock.vn/BID/tai-chinh.htm?tab=CSTC")
time.sleep(5)
driver.find_element(By.ID, "txtEmailLogin").send_keys("foreveralone1789@gmail.com")
pw = driver.find_element(By.ID, "passwordLogin")
pw.send_keys("Thanhdang@19")
pw.send_keys(Keys.RETURN)
time.sleep(5)

# Duyệt qua từng ngân hàng và tab
for bank in banks:
    os.makedirs(bank, exist_ok=True)
    print(f"\n🔄 {bank}")

    for t in tabs:
        driver.get(f"https://finance.vietstock.vn/{bank}/tai-chinh.htm?tab={t['tab']}")
        time.sleep(5)

        try:
            Select(driver.find_element(By.NAME, "period")).select_by_value("-1")
            Select(driver.find_element(By.NAME, "UnitDong")).select_by_value("1000000")
            time.sleep(10)  # Chờ bảng load
        except:
            pass

        try:
            table = driver.find_element(By.ID, t["table_id"])
            df = pd.read_html(table.get_attribute("outerHTML"))[0]
            df.to_excel(os.path.join(bank, f"{t['tab']}.xlsx"), index=False)
            print(f"✅ {t['tab']}")
        except Exception as e:
            print(f"❌ {t['tab']} lỗi: {e}")

driver.quit()


🔄 VPB


C:\Users\THANH DANG\AppData\Local\Temp\ipykernel_26008\3384099298.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


✅ BCTT


C:\Users\THANH DANG\AppData\Local\Temp\ipykernel_26008\3384099298.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


✅ CDKT


...

✅ LC


C:\Users\THANH DANG\AppData\Local\Temp\ipykernel_26008\3384099298.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


✅ CSTC


C:\Users\THANH DANG\AppData\Local\Temp\ipykernel_26008\3384099298.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


✅ TM


In [12]:
import os
import pandas as pd

# === THƯ MỤC GỐC: THƯ MỤC HIỆN TẠI CHỨA CÁC FOLDER NGÂN HÀNG ===
root_dir = os.getcwd()

all_data = []

for bank_name in os.listdir(root_dir):
    bank_path = os.path.join(root_dir, bank_name)
    if not os.path.isdir(bank_path):
        continue

    for filename in os.listdir(bank_path):
        if not filename.endswith((".xlsx", ".xls")):
            continue

        file_path = os.path.join(bank_path, filename)
        source_name = os.path.splitext(filename)[0]  # bỏ đuôi .xlsx/.xls

        try:
            xls = pd.ExcelFile(file_path)
            for sheet_name in xls.sheet_names:
                try:
                    df_raw = xls.parse(sheet_name, header=None)

                    time_row_index = df_raw.index[
                        df_raw.apply(lambda row: row.astype(str).str.contains(r"\d{4}").any(), axis=1)
                    ].min()
                    df = df_raw.iloc[time_row_index:]

                    df.columns = df.iloc[0]
                    df = df[1:]
                    
                    df = df.set_index(df.columns[0])
                    df = df.transpose().reset_index()
                    df = df.rename(columns={"index": "Time"})

                    df.insert(0, "Bank", bank_name)
                    df.insert(2, "Source", source_name)  # ← dùng tên file ở đây

                    all_data.append(df)

                except Exception as e:
                    print(f"[!] Lỗi đọc sheet: {sheet_name} trong file {filename}: {e}")

        except Exception as e:
            print(f"[!] Lỗi mở file Excel: {file_path}: {e}")

df_all = pd.concat(all_data, ignore_index=True)

output_path = os.path.join(root_dir, "BaoCao_TongHop_Financials.xlsx")
df_all.to_excel(output_path, index=False)

print(f"✅ Đã lưu file tổng hợp tại: {output_path}")

✅ Đã lưu file tổng hợp tại: c:\Users\THANH DANG\Desktop\New folder (4)\BaoCao_TongHop_Financials.xlsx


In [3]:
import os
import pandas as pd

# Đường dẫn thư mục gốc chứa các thư mục ngân hàng
ROOT_DIR = "./"
BAO_CAO_TYPES = ["BCTT.xlsx", "CDKT.xlsx", "CSTC.xlsx", "KQKD.xlsx", "LC.xlsx", "TM.xlsx"]

# Tạo thư mục output nếu chưa tồn tại
OUTPUT_FOLDER = "Tổng hợp ngân hàng"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def gop_bao_cao(bc_filename):
    data_list = []

    for bank_folder in os.listdir(ROOT_DIR):
        bank_path = os.path.join(ROOT_DIR, bank_folder)
        file_path = os.path.join(bank_path, bc_filename)

        if os.path.isdir(bank_path) and os.path.exists(file_path):
            try:
                df = pd.read_excel(file_path, sheet_name=None)
                for sheet_name, sheet_df in df.items():
                    sheet_df.columns = sheet_df.columns.astype(str)
                    # Cột đầu là chỉ tiêu
                    chi_tieu = sheet_df.iloc[:, 0]
                    time_cols = [col for col in sheet_df.columns if col != sheet_df.columns[0]]

                    for col in time_cols:
                        values = sheet_df[col].values
                        row = {"Bank": bank_folder, "Quý": col}
                        row.update(dict(zip(chi_tieu, values)))
                        data_list.append(row)

            except Exception as e:
                print(f"Lỗi đọc {file_path}: {e}")

    if not data_list:
        print(f"❗ Không có dữ liệu cho báo cáo {bc_filename}")
        return

    df_final = pd.DataFrame(data_list)
    output_path = os.path.join(OUTPUT_FOLDER, f"{bc_filename.replace('.xlsx', '')}_TongHop.xlsx")
    df_final.to_excel(output_path, index=False)
    print(f"✅ Đã gộp xong: {bc_filename} → {output_path}")

# Gộp tất cả các báo cáo
for bc_file in BAO_CAO_TYPES:
    gop_bao_cao(bc_file)

✅ Đã gộp xong: BCTT.xlsx → Tổng hợp ngân hàng\BCTT_TongHop.xlsx
✅ Đã gộp xong: CDKT.xlsx → Tổng hợp ngân hàng\CDKT_TongHop.xlsx
✅ Đã gộp xong: CSTC.xlsx → Tổng hợp ngân hàng\CSTC_TongHop.xlsx
✅ Đã gộp xong: KQKD.xlsx → Tổng hợp ngân hàng\KQKD_TongHop.xlsx
✅ Đã gộp xong: LC.xlsx → Tổng hợp ngân hàng\LC_TongHop.xlsx
✅ Đã gộp xong: TM.xlsx → Tổng hợp ngân hàng\TM_TongHop.xlsx
